In [1]:
from langchain_core.tools import tool

In [2]:
import requests

In [3]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

In [4]:

@tool 
def conv_fact(base_curr:str,target_curr:str)->float:
    '''this function is used to fetch the currency conversion 
    factor between base currency and target currency'''
    url = f"https://v6.exchangerate-api.com/v6/9c322631415235554a6c9be7/pair/{base_curr}/{target_curr}"
    response = requests.get(url)
    if response.status_code != 200:
        print("Error fetching data:", response.status_code, response.text)
        return {"error": "Failed to fetch conversion rate"}
    return response.json()

@tool
def conv(base_curr:int,conversion_rate:Annotated[float,InjectedToolArg])->float:
    #this enables to fill the conv_rate by user only not llm pre trained value
    '''given a currency conversion rate this function caluclates the 
    target currency value from the base currency value'''

    return base_curr* conversion_rate

In [5]:
conv.invoke({'base_curr':5,'conversion_rate':85.5})

427.5

In [6]:
from langchain_ollama import ChatOllama

In [7]:
model = ChatOllama(
    model='qwen2.5:3b'
)

In [8]:
llm_with_tools = model.bind_tools([conv_fact,conv])

In [9]:
from langchain_core.messages import HumanMessage

In [10]:
messages = [HumanMessage('what is the conversion factor between USD to INR and convert 50 USD to INR')]

In [11]:
ai_msg = llm_with_tools.invoke(messages)

In [12]:
messages.append(ai_msg)

In [13]:
ai_msg.tool_calls

[{'name': 'conv_fact',
  'args': {'base_curr': 'USD', 'target_curr': 'INR'},
  'id': 'a8ac2ef6-fa1a-4498-9f1f-6eb206063b76',
  'type': 'tool_call'},
 {'name': 'conv',
  'args': {'base_curr': 50},
  'id': 'eb9b10b2-fcd4-4728-a463-5aa8788e65af',
  'type': 'tool_call'}]

In [14]:
import json
for tool_call in ai_msg.tool_calls:
    if tool_call['name'] == 'conv_fact':
        print(tool_call)
        tool_message1 = conv_fact.invoke(tool_call)
        conv_rate = json.loads(tool_message1.content)['conversion_rate']
        messages.append(tool_message1)

    if tool_call['name'] == 'conv':
        tool_call['args']['conversion_rate'] = conv_rate
        messages.append(conv.invoke(tool_call))
        
    
   

{'name': 'conv_fact', 'args': {'base_curr': 'USD', 'target_curr': 'INR'}, 'id': 'a8ac2ef6-fa1a-4498-9f1f-6eb206063b76', 'type': 'tool_call'}


In [15]:
messages

[HumanMessage(content='what is the conversion factor between USD to INR and convert 50 USD to INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:3b', 'created_at': '2025-09-18T18:23:29.5856782Z', 'done': True, 'done_reason': 'stop', 'total_duration': 62394041500, 'load_duration': 31084483700, 'prompt_eval_count': 265, 'prompt_eval_duration': 21706120900, 'eval_count': 74, 'eval_duration': 9598835500, 'model_name': 'qwen2.5:3b'}, id='run--b562da5d-6d51-4b96-b34e-d405f28918fc-0', tool_calls=[{'name': 'conv_fact', 'args': {'base_curr': 'USD', 'target_curr': 'INR'}, 'id': 'a8ac2ef6-fa1a-4498-9f1f-6eb206063b76', 'type': 'tool_call'}, {'name': 'conv', 'args': {'base_curr': 50, 'conversion_rate': 87.861}, 'id': 'eb9b10b2-fcd4-4728-a463-5aa8788e65af', 'type': 'tool_call'}], usage_metadata={'input_tokens': 265, 'output_tokens': 74, 'total_tokens': 339}),
 ToolMessage(content='{"result": "success", "documentation":

In [16]:
res = llm_with_tools.invoke(messages)

In [18]:
res.content

'The conversion factor between USD to INR is approximately 87.861 (as retrieved from the exchange rate API). \n\nBased on this, 50 USD converts to approximately 4393.05 INR. Please note that these values are based on the last update date of Thu, 18 Sep 2025 00:00:01 +0000 and will be subject to change as per the terms of use provided by the exchange rate API at https://www.exchangerate-api.com/terms.'